In [19]:
import numpy as np
import pandas as pd
import requests
import json
import config
import sqlalchemy
from sqlalchemy import Table, Column, Integer, String, MetaData


## ETL Pipeline

In [2]:
def google_books_call(query, api_key, n_calls):
    data = []
    
    while n_calls > 0:
        url = f'https://www.googleapis.com/books/v1/volumes?q={query}=full&maxResults=40&key={config.googlebooks_api_key}'
        response = requests.request('GET', url, allow_redirects=False)
        load = json.loads(response.text)

        data.append(load['items'])
        n_calls -=1
    return data


In [38]:
data = google_books_call('computer programming', config.googlebooks_api_key, 1)

In [41]:
data[0][0]['volumeInfo']

{'title': "The Complete Idiot's Guide to a Career in Computer Programming",
 'authors': ['Jesse Liberty'],
 'publisher': 'Penguin',
 'publishedDate': '1999',
 'description': 'Describes the job market, qualifications, career paths, and common pitfalls and includes information on interviewing, working with employment agencies, and resumes',
 'industryIdentifiers': [{'type': 'ISBN_10', 'identifier': '0789719959'},
  {'type': 'ISBN_13', 'identifier': '9780789719959'}],
 'readingModes': {'text': False, 'image': True},
 'pageCount': 282,
 'printType': 'BOOK',
 'categories': ['Computers'],
 'maturityRating': 'NOT_MATURE',
 'allowAnonLogging': False,
 'contentVersion': '1.0.1.0.preview.1',
 'panelizationSummary': {'containsEpubBubbles': False,
  'containsImageBubbles': False},
 'imageLinks': {'smallThumbnail': 'http://books.google.com/books/content?id=IzvDMZURXQMC&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api',
  'thumbnail': 'http://books.google.com/books/content?id=IzvDMZURXQMC&p

In [5]:
data[0][0]['saleInfo']

{'country': 'US',
 'saleability': 'FOR_SALE',
 'isEbook': True,
 'listPrice': {'amount': 79.99, 'currencyCode': 'USD'},
 'retailPrice': {'amount': 63.99, 'currencyCode': 'USD'},
 'buyLink': 'https://play.google.com/store/books/details?id=omivDQAAQBAJ&rdid=book-omivDQAAQBAJ&rdot=1&source=gbs_api',
 'offers': [{'finskyOfferType': 1,
   'listPrice': {'amountInMicros': 79990000.0, 'currencyCode': 'USD'},
   'retailPrice': {'amountInMicros': 63990000.0, 'currencyCode': 'USD'},
   'giftable': True}]}

In [42]:
vol_keys = ['title', 'subtitle', 'authors', 'averageRating', 'ratingsCount', 'categories',
       'readingModes', 'pageCount', 'publishedDate', 'description']
sale_keys = ['isEbook', 'listPrice']

def filter_keys(data, keys):
    return{key: data[key] if key in data.keys() else None for key in keys}

In [43]:
def parse_books(data, keys1, keys2):
    volumes = []
    for item in data:
    #     print(item)
        for book in item:
#             print(book)
            volInfo = book['volumeInfo']
            voldict = filter_keys(keys=keys1, data=volInfo)
            if type(voldict['readingModes']) is dict:
                    voldict['readingModes'] = voldict['readingModes']['image']
            saleInfo = book['saleInfo']
            saledict = filter_keys(keys=keys2, data=saleInfo)
            if type(saledict['listPrice']) is dict:
                saledict['listPrice'] = saledict['listPrice']['amount']
            bigdict = {**voldict, **saledict}
            volumes.append(bigdict)
    return volumes

In [44]:
df = pd.DataFrame(parse_books(data, vol_keys, sale_keys))
df.head()

,title,subtitle,authors,averageRating,ratingsCount,categories,readingModes,pageCount,publishedDate,description,isEbook,listPrice
0,The Complete Idiot's Guide to a Career in Comp...,None,[Jesse Liberty],NaN,NaN,[Computers],True,282.0,1999,"Describes the job market, qualifications, care...",False,NaN
1,Structure and Interpretation of Computer Programs,None,[Harold Abelson],5.0,3.0,[Computers],False,657.0,1996,Structure and Interpretation of Computer Progr...,False,NaN
2,Part Time Computer Programmer Full Time Dad No...,Programming Notebook Journal Blanked Lined Rul...,[Hab Publication],NaN,NaN,None,False,124.0,2019-07,A Gift for Programming. This is Remarkable and...,False,NaN
3,Computer Programming and Computer Systems,None,[Anthony Hassitt],NaN,NaN,[Business & Economics],True,384.0,2014-05-12,Computer Programming and Computer Systems impa...,True,72.95
4,The Art of Computer Programming,None,[Donald Ervin Knuth],NaN,NaN,[Computers],True,134.0,2005,"Author's pref. : ""the first of a series of upd...",False,NaN


In [45]:
df = df[~df.categories.isnull()]
df['authors'] = df['authors'].apply(lambda x: ', '.join(x))
df['categories'] = df['categories'].apply(lambda x: ', '.join(x))
df.rename(columns={"readingModes": "images"}, inplace=True)
df.head()

,title,subtitle,authors,averageRating,ratingsCount,categories,images,pageCount,publishedDate,description,isEbook,listPrice
0,The Complete Idiot's Guide to a Career in Comp...,None,Jesse Liberty,NaN,NaN,Computers,True,282.0,1999,"Describes the job market, qualifications, care...",False,NaN
1,Structure and Interpretation of Computer Programs,None,Harold Abelson,5.0,3.0,Computers,False,657.0,1996,Structure and Interpretation of Computer Progr...,False,NaN
3,Computer Programming and Computer Systems,None,Anthony Hassitt,NaN,NaN,Business & Economics,True,384.0,2014-05-12,Computer Programming and Computer Systems impa...,True,72.95
4,The Art of Computer Programming,None,Donald Ervin Knuth,NaN,NaN,Computers,True,134.0,2005,"Author's pref. : ""the first of a series of upd...",False,NaN
5,Computer Science,An Overview,J. Glenn Brookshear,4.5,2.0,Computer science,False,609.0,2000-01,This title provides broad coverage and clear e...,False,NaN


In [46]:
df.categories.value_counts()

Computers               25
Juvenile Nonfiction      2
Occupations              1
Education                1
Computer science         1
Algorithms               1
Business & Economics     1
Name: categories, dtype: int64

In [47]:
df.listPrice.isnull().sum()

25

In [48]:
df[df['ratingsCount'] >= 2]

,title,subtitle,authors,averageRating,ratingsCount,categories,images,pageCount,publishedDate,description,isEbook,listPrice
1,Structure and Interpretation of Computer Programs,None,Harold Abelson,5.0,3.0,Computers,False,657.0,1996,Structure and Interpretation of Computer Progr...,False,NaN
5,Computer Science,An Overview,J. Glenn Brookshear,4.5,2.0,Computer science,False,609.0,2000-01,This title provides broad coverage and clear e...,False,NaN
7,The Art of Computer Programming,Volume 1: Fundamental Algorithms,Donald E. Knuth,5.0,4.0,Computers,True,672.0,1997-07-04,The bible of all fundamental algorithms and th...,False,NaN
8,"Art of Computer Programming, Volume 2",Seminumerical Algorithms,Donald E. Knuth,5.0,2.0,Computers,True,784.0,2014-05-06,The bible of all fundamental algorithms and th...,False,NaN
9,"Concepts, Techniques, and Models of Computer P...",None,"Peter Van-Roy, Seif Haridi",3.5,6.0,Computers,True,900.0,2004,Running the Example Programs - Introduction to...,False,NaN
12,Computer Science: A Very Short Introduction,None,Subrata Dasgupta,4.0,3.0,Computers,True,144.0,2016-03-07,"Over the past sixty years, the spectacular gro...",False,NaN
15,How to Design Programs,An Introduction to Programming and Computing,"Matthias Felleisen, Robert Bruce Findler, Shri...",4.5,8.0,Computers,True,693.0,2001,Processing simple forms of data - Processing a...,False,NaN
17,Fundamentals of Computer Programming with C#,The Bulgarian C# Book,"Svetlin Nakov, Veselin Kolev",5.0,2.0,Computers,True,1132.0,2013-09-01,"The free book ""Fundamentals of Computer Progra...",False,NaN
18,A Complete Guide to Programming in C++,None,"Ulla Kirch-Prinz, Peter Prinz",4.5,4.0,Computers,True,825.0,2002,A Complete Guide to Programming in C++ was wri...,False,NaN
20,"Art of Computer Programming, Volume 4, Fascicl...",Generating All Trees--History of Combinatorial...,Donald E. Knuth,4.0,5.0,Computers,True,128.0,2013-09-25,"Finally, after a wait of more than thirty-five...",True,19.99


In [14]:
engine = sqlalchemy.create_engine(f'mysql://{config.user}:{config.password}@{config.host}')
engine.execute("CREATE DATABASE books")
engine.execute("USE books")

In [35]:
meta = MetaData()

books = Table(
    'books', meta, 
    Column('index', Integer, primary_key = True), 
    Column('title', String), 
    Column('subtitle', String), 
    Column('authors', String),
    Column()
)

meta.create_all(engine)